In [ ]:
import streamlit as st
from crewai import Crew, Process
from travel_agents import guide_expert, location_expert, planner_expert
from travel_tasks import guide_task, location_task, planner_task

st.title('AI - Powered Trip Planner')

st.markdown(""" 
            *** Plan your next trip with AI !***
            Enter your travel details below and our AI-Powered travel assistant will create a personalized itinerary including:
            Best plances to visit.   Accommodation & budget planning.  
            Local food recommendations.   Transportation & Visa details
            """)

from_city = st.text_input('From City', 'India')
destination_city = st.text_input('Destination City', 'Paris')
date_from = st.date_input('Departure Date')
date_to = st.date_input('Return Date')
interests = st.text_area('Your Interests (e.g. Sightseeing, food, adventure, local festival)', 'sightseeing and good food')

if st.button(' Generate Travel Plan'):
    if not from_city or not destination_city or not date_from or not date_to or not interests:
        st.error('Please fill in all fields before generating your travel plan.')
    else:
        st.write('AI is preparing your personalized travel itinerary.... Please wait. ')

        loc_task = location_task(location_expert, from_city, destination_city, date_from, date_to)
        guid_task = guide_task(guide_expert, destination_city, interests, date_from, date_to)
        plan_task = planner_task([loc_task, guid_task], planner_expert, destination_city, interests, date_from, date_to)

        crew = Crew(
            agents = [location_expert, guide_expert, planner_expert],
            tasks = [loc_task, guid_task, plan_task],
            process = Process.sequential,
            full_output = True,
            verbose = True,
        )

        result = crew.kickoff()

        st.subheader('Your AI-Powered Travel Plan')
        st.markdown(result)

        travel_plan_text = str(result)

        st.download_button(
            label = 'Download Travel Plan',
            data = travel_plan_text,
            file_name=f'Travel_Plan_{destination_city}.txt',
            mime='text/plain'
        )
